In [1]:
# Imports
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt
import requests
import os
import datetime as dt
from datetime import timedelta, date

In [2]:
df_1 = pd.read_csv('Data/Oct_Dec_2022_Lethbridge_Police_Service_report.csv')
df_2 = pd.read_csv('Data/Jan_Mar_2023_Lethbridge_Police_Service_report.csv')
df_3 = pd.read_csv('Data/Apr_Jun_2023_Lethbridge_Police_Service_report.csv')

df = pd.concat([df_1, df_2, df_3])
df = df.drop_duplicates()
df.head()

,ccn,date,updateDate,city,state,postalCode,blocksizedAddress,incidentType,parentIncidentType,narrative
0,CA23018227,"12/25/2022, 12:00:00 AM","08/03/2023, 1:23:51 PM",LETHBRIDGE,AB,.,1600 Block ST GEORGE RD,RMS] Assault,Assault,Assault
1,CA23016523,"12/01/2022, 9:04:00 AM","08/03/2023, 8:23:45 AM",LETHBRIDGE,AB,T1J 5J4,100 Block GOLDENROD RD,RMS] Fraud - Identity Fraud,Other,Fraud - Identity Fraud
2,RM23054146,"10/21/2022, 4:00:00 PM","07/22/2023, 8:39:56 AM",LETHBRIDGE,AB,.,100 Block FAIRMONT BD,RMS] Proceeds Of Crime Criminal Cod,Other,Proceeds Of Crime Criminal Cod
3,CA23008687,"11/01/2022, 7:00:00 AM","07/06/2023, 9:00:30 AM",LETHBRIDGE,AB,.,1 Block BLACKFOOT CI,RMS] Sexual Assault,Sexual Offense,Sexual Assault
4,RM22058254,"12/16/2022, 1:00:00 PM","07/06/2023, 9:00:20 AM",LETHBRIDGE,AB,.,100 Block 1 AV,RMS] Assist Others,Other,Assist Others


In [3]:
df.describe()

,ccn,date,updateDate,city,state,postalCode,blocksizedAddress,incidentType,parentIncidentType,narrative
count,68538,68538,68538,68538,68538,68538,68526,68538,68538,68538
unique,68538,65515,14349,31,3,226,2990,536,13,289
top,CA23018227,"02/01/2023, 3:00:00 PM","11/24/2022, 1:24:20 AM",LETHBRIDGE,AB,.,UNKNOWN STREET,PUBLIC SERVICE,Other,PUBLIC SERVICE
freq,1,17,60,68451,68535,67657,3691,5772,37446,5772


There are 31 unique values for the 'city' column, and there should only be 1 ('LETHBRIDGE').  Looking at the unique values we can see that there are some incidents that are in other cities and should not be included.  There is also 1 incident with a typo ('LETHBRIDE'), and this needs to be corrected.

In [4]:
df['city'].value_counts()

city
LETHBRIDGE           68451
COALDALE                15
LETHBRIDGE COUNTY       14
COALHURST                8
FORT MACLEOD             8
CALGARY                  5
TABER                    4
CARDSTON                 4
RAYMOND                  3
FAIRVIEW                 3
PINCHER CREEK            2
STAND OFF                2
BELLEVUE                 1
DR                       1
BARONS                   1
COUNTY LETHBRIDGE        1
WETASKIWIN               1
OTTAWA                   1
LAVAL                    1
BROOKS                   1
THREE HILLS              1
RED DEER                 1
SAINT-LAURENT            1
Lethbridge               1
EDMONTON                 1
RED CLIFF                1
CLARESHOLM               1
MAGRATH                  1
STIRLING                 1
LETHBRIDE                1
NOBLEFORD                1
Name: count, dtype: int64

In [5]:
# Fixing typos and dropping any rows where the city is not 'LETHBRIDGE'
df = df.replace('LETHBRIDE', 'LETHBRIDGE')
df = df[df['city'] == 'LETHBRIDGE']

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 68452 entries, 0 to 24689
Data columns (total 10 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   ccn                 68452 non-null  object
 1   date                68452 non-null  object
 2   updateDate          68452 non-null  object
 3   city                68452 non-null  object
 4   state               68452 non-null  object
 5   postalCode          68452 non-null  object
 6   blocksizedAddress   68446 non-null  object
 7   incidentType        68452 non-null  object
 8   parentIncidentType  68452 non-null  object
 9   narrative           68452 non-null  object
dtypes: object(10)
memory usage: 5.7+ MB


There are some null values in the 'blocksizedAddress' column, let's remove these

In [7]:
df = df.dropna(subset=['blocksizedAddress'], axis=0)

Let's look at the values in the 'parentIncidentType' column

In [8]:
df['parentIncidentType'].value_counts()

parentIncidentType
Other                  37376
Quality of Life         9761
Proactive Policing      9408
Traffic                 4143
Theft                   3198
Assault                 1664
Theft from Vehicle      1437
Property Crime           677
Sexual Offense           447
Theft of Vehicle         244
Robbery                   79
Breaking & Entering       10
Homicide                   2
Name: count, dtype: int64

Let's remove all 'Proactive Policing' and 'Traffic' incidents, as we only want crimes reported by the public.  'Proactive Policing' and 'Traffic' incidents are usually only registered when witnessed and acted upon by police.  There is a possibility that some 'Traffic" incidents have been reported by the public, but we'll assume that this is a very small number.

In [9]:
df = df[~df['parentIncidentType'].isin(['Proactive Policing', 'Traffic'])]
df.describe()

,ccn,date,updateDate,city,state,postalCode,blocksizedAddress,incidentType,parentIncidentType,narrative
count,54895,54895,54895,54895,54895,54895,54895,54895,54895,54895
unique,54895,52300,13396,1,1,82,2469,500,11,276
top,CA23018227,"02/01/2023, 3:00:00 PM","11/24/2022, 1:24:18 AM",LETHBRIDGE,AB,.,100 Block 1 AV,PUBLIC SERVICE,Other,PUBLIC SERVICE
freq,1,17,54,54895,54895,54586,2193,5772,37376,5772


We don't need the 'updateDate', 'incidentType' or 'postalCode' columns so let's drop them

In [10]:
df = df.drop(['updateDate','incidentType', 'postalCode'], axis=1)

Most of the values in the 'blocksizedAddress' refer to a city block, which is what we want. However some refer to a street corner and others are 'UNKNOWN STREET'.  Let's drop these ones for the sake of simplicity.

In [11]:
df = df[df['blocksizedAddress'].str.contains('Block')]

Now we'll make a new column for the full address

In [12]:
df['address'] = df['blocksizedAddress'].str.replace('Block ', '') + ', ' + df['city'] + ', ' + df['state'] + ', Canada'
df[['address']]

,address
0,"1600 ST GEORGE RD, LETHBRIDGE, AB, Canada"
1,"100 GOLDENROD RD, LETHBRIDGE, AB, Canada"
2,"100 FAIRMONT BD, LETHBRIDGE, AB, Canada"
3,"1 BLACKFOOT CI, LETHBRIDGE, AB, Canada"
4,"100 1 AV, LETHBRIDGE, AB, Canada"
...,...
24682,"300 8 ST, LETHBRIDGE, AB, Canada"
24683,"1 LAFAYETTE BD, LETHBRIDGE, AB, Canada"
24687,"800 2A AV, LETHBRIDGE, AB, Canada"
24688,"100 LAVAL BD, LETHBRIDGE, AB, Canada"


Now we'll create a pivot table with each unique address as a row, and save it to csv file.  We will find the latitudes and longitudes in the 2nd notebook using the Google Geocoding API.

In [19]:
df_address = df.pivot_table(index='address', values='ccn', aggfunc='count').reset_index()
df_address.columns = ['address', 'incidentCount']
df_address.to_csv('df_address.csv', index=False)

And we'll also write the incident dataframe to a csv file:

In [ ]:
df.to_csv('df.csv', index=False)

# END